# Tensorflow-Estimator-自定义估算器
这篇文章介绍自定义一个估算器（分类器）Estimator的完整流程。

## 自定义Custom Estimator和预制Pre-made Estimator
在上面iris的案例中我们使用了tensorflow里面自带的深度神经网络分类器tf.estimator.DNNClassifie。这些tensorflow自带的estimator称为预制估算器Pre-made Estimator（预创建的Estimator）。

Tensorflow允许我们自己创建更加灵活的Custom Estimator。自定义Estimator是tf.estimator.Estimator()方法生成，能够像预制Estimator一样使用。


## 结构概览

从表面看，我们的Estimator应该具有DNNClassifier一样的功能

创建的时候接收一些参数，如feature_columns、hidden_units、n_classes等

具有train()、evaluate()、predict()三个方法用来训练、评价、预测

如上所说，我们使用 tf.estimator.Estimator()方法来生成自定义Estimator，它的语法格式是



模型函数model_fn是唯一没有默认值的参数，它也是自定义Estimator最关键的部分，包含了最核心的
算法。model_fn需要一个能够进行运算的函数，它的样子应该长成这样


## 神经网络层Layers

输入层Input Layer，数据从这里进入

隐藏层Hidden Layer，2个，每层包含多个节点，数据流经这里，被推测规律

输出层Output Layer，将推测的结果整理显示出来

我们并不需要手工实现隐藏层的算法和工作原理，Tensorflow已经为我们设计好。我们需要的只是创建这些神经网络层，并确保它们按照正常的顺序连接起来，至于其中如何推算演绎的魔法就完全交给tensorflow就可以了。

mode_fn需要完成的就是创建和组织这些神经层。



## 编写model_fn
对应我们创建Estimator时候的参数

这些参数都会被Estimator打包放在params超参数中，传递给model_fn，所以我们用下面的代码在model_fn内创建网络层

## 输入层Input Layer
Input Layer把输入的数据features填充到特征列params['feature_column']里面,稍后它会被继续传递到隐藏层hidden layer

## 隐藏层Hidden Layer
我们使用循环为hidden_unit列表([10,10])创建了2个隐藏图层，每个图层的神经元节点unit都等于10.

## 输出层Output Layer
仍然是链条的延续！

但是activation这里改为了None，不再激活后续的部分，所以输出层就是链条的终点。

## 训练train、评价evaluate和预测predict
前面我们知道，自定义的估算分类器必须能够用来执行my_classifier.train()、my_classifier.evaluate()、my_classifier.predict()三个方法。

### 但实际上，它们都是model_fn这一个函数的分身！

上面出现的model_fn语法：

第三个参数mode，如果它等于"TRAIN"我们就执行训练：

如果是“EVAL”就执行评价，“PREDICT”就执行预测。

## 预测Predict
因为预测最后我们需要返回花的种类label，还希望知道这个预测有多精确，所以在预测部分的代码里面，首先取到三种花可能性最大的一个predicted_classes即[-1.3,2.6,-0.9]中的2.6；然后把它转成列表格式[[2.6]];同时把logit得到的[-1.3,2.6,-0.9]转化为表示0～1可能性的小数[0.01926995 0.95198274 0.02874739]


##### 注意最后一句，我们返回return的是一个EstimatorSpec对象，下面的训练predict和评价evaluate也都返回EstimatorSpec形式的对象，但是参数不同，请留意。

我们可以使用以下代码在单独文件测试tf.newaxis和tf.nn.softmax对数据转化的作用

输出

## 损失函数Loss

损失函数是Tensorflow中神经网络的重要概念，简单说，它能够计算出我们模型的偏差程度，结果越大，我们的模型就偏差越大、离正确也远、也越不准确、越糟糕。

为了降低损失，我们可以使用更多更好的数据，还可以设计更好的优化方法，来优化改进模型，让损失变为最小。

训练神经网络模型的目标就是把偏差损失降为最小，机器学习就是一批一批数据反复分析计算反复尝试，不断的利用优化方法，想尽办法把Loss的值降到最小的过程。

优化方法设计的越好好，损失也就越少，精度也就越高。


这里我们使用了tensorflow提供的稀疏柔性最大交叉熵sparse_softmax_cross_entropy来计算损失程度，它对于分类问题很有效，DNNClassifier也使用了这个方法。

## 训练Train
我们在训练部分代码中，创建了优化器optimizer，然后使用它尝试将我们的损失函数loss变为最小minimize：

## 评价Evaluate
我们使用下面的代码来评价预测结果prediction和test数据中植物学家标记的数据是否足够吻合:

# 完整程序如下：

In [1]:
import tensorflow as tf
import pandas as pd
import os

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 导入数据

In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL =  "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES =  ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def load_data(label_name = 'Species'):
    train_path = tf.keras.utils.get_file(
        fname=os.path.basename(TRAIN_URL),
        origin=TRAIN_URL)
    train = pd.read_csv(
        train_path,
        names = CSV_COLUMN_NAMES,
        header = 0
    )
    train_features,train_label = train,train.pop(label_name)
    
    test_path = tf.keras.utils.get_file(
        fname=os.path.basename(TEST_URL),
        origin=TEST_URL
    )
    test = pd.read_csv(
        test_path,
        names = CSV_COLUMN_NAMES,
        header = 0
    )
    test_features,test_label = test,test.pop(label_name)
    
    return (train_features,train_label),(test_features,test_label)

In [3]:
(train_x,train_y),(test_x,test_y) = load_data()
print(train_x[:5])

   SepalLength  SepalWidth  PetalLength  PetalWidth
0          6.4         2.8          5.6         2.2
1          5.0         2.3          3.3         1.0
2          4.9         2.5          4.5         1.7
3          4.9         3.1          1.5         0.1
4          5.7         3.8          1.7         0.3


## 特征列

In [4]:
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key))

## 编写model_fn

In [5]:
def my_model(features,labels,mode,params):
    #输入层,feature_columns对应Classifier(feature_columns=...)
    net = tf.feature_column.input_layer(features,params['feature_columns'])
    
    #隐藏层,hidden_units对应Classifier(unit=[10,10])，2个各含10节点的隐藏层
    for units in params['hidden_units']:
        net = tf.layers.dense(net,units=units,activation=tf.nn.relu)
    
    #输出层，n_classes对应3种鸢尾花
    logits = tf.layers.dense(net,units=params['n_classes'],activation=None)
    #预测
    predicted_classes = tf.argmax(logits,1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids':predicted_classes[:,tf.newaxis],
            'probabilities':tf.nn.softmax(logits),
            'logits':logits,
        }
        return tf.estimator.EstimatorSpec(mode,predictions=predictions)
    #损失函数
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,logits=logits)
    
    #训练
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.1) #用它优化损失函数，达到损失最少精度最高
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())  #执行优化！
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

   #评价
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op') #计算精度
    metrics = {'accuracy': accuracy} #返回格式
    tf.summary.scalar('accuracy', accuracy[1]) #仅为了后面图表统计使用
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)


## 创建分类器

In [6]:
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    model_dir = 'my_dir/iris',
    params = {
        'feature_columns': my_feature_columns,
        'hidden_units':[10,10],
        'n_classes':3,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'my_dir/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000017AC4932DA0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 训练模型

In [7]:
#针对训练的喂食函数
batch_size=100
def train_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size) #每次随机调整数据顺序
    return dataset.make_one_shot_iterator().get_next()

#开始训练
classifier.train(
    input_fn=lambda:train_input_fn(train_x, train_y, 100),
    steps=1000)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from my_dir/iris\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3000 into my_dir/iris\model.ckpt.
INFO:tensorflow:loss = 0.022829015, step = 3001
INFO:tensorflow:global_step/sec: 520.567
INFO:tensorflow:loss = 0.04111736, step = 3101 (0.208 sec)
INFO:tensorflow:global_step/sec: 861.816
INFO:tensorflow:loss = 0.038586494, step = 3201 (0.100 sec)
INFO:tensorflow:global_step/sec: 961.877
INFO:tensorflow:loss = 0.01956015, step = 3301 (0.104 sec)
INFO:tensorflow:global_step/sec: 961.167
INFO:tensorflow:loss = 0.04609591, step = 3401 (0.108 sec)
INFO:tensorflow:global_step/sec: 924.674
INFO:tensorflow:loss = 0.059080057, step = 3501 (0.104 sec)
INFO:tensorflow:global_step/sec: 961.416
INFO:tensorflow:loss = 0.0493

## 评价模型

In [8]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    inputs = (features,labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

#评价结果
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(test_x,test_y,batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-06-00:25:49
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from my_dir/iris\model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-06-00:25:50
INFO:tensorflow:Saving dict for global step 4000: accuracy = 0.96666664, global_step = 4000, loss = 0.05673013
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 4000: my_dir/iris\model.ckpt-4000

Test set accuracy: 0.967



## 预测

In [ ]:
#支持100次循环对新数据进行分类预测
for i in range(0,100):
    print('\nPlease enter features: SepalLength,SepalWidth,PetalLength,PetalWidth')
    a,b,c,d = map(float, input().split(',')) #捕获用户输入的数字
    predict_x = {
        'SepalLength': [a],
        'SepalWidth': [b],
        'PetalLength': [c],
        'PetalWidth': [d],
    }
    
    #进行预测
    predictions = classifier.predict(
        input_fn=lambda:eval_input_fn(predict_x,
                                      labels=[0,],
                                      batch_size=batch_size))    

    #预测结果是数组，尽管实际我们只有一个
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]
        print(SPECIES[class_id],100 * probability)



Please enter features: SepalLength,SepalWidth,PetalLength,PetalWidth
